<a href="https://colab.research.google.com/github/febuz/MADspace.github.io/blob/master/DEFINITIONS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DEFINITIONS
Here we developed the 

-OLS Regression 

-Correlation functions (preparation AR models)

-Candlestick definitions

-Augmented Dickey Fuller stationair test 

for JHTALIB
#Regression
Regression formula:

Source: https://en.wikipedia.org/wiki/Autoregressive_model

Source: https://www.fmlabs.com/reference/RegressionLineMv.htm

Source: https://github.com/statsmodels/statsmodels/

#Ordinairy Least Squares Regression
1. check the stationality of the time series 
2. test the stationality of the data

    a) Dickey-Fuller test for unit root
    
    b) Augmented Dickey-Fuller (ADF) test for unit root
3. only use stationairy series for your model, normalize or standardize other wise 
4. Basis OLS model: 

y = mx + b

y = how far up

x = how far along

m = Slope or Gradient (how steep the line is)

b = the Y Intercept (where the line crosses the Y axis)

5. find the line of best fit for a group of (x,y) points:

Step 1:	For each (x,y) calculate x2 and xy

Step 2:	Sum all x, y, x2 and xy, which gives us Σx, Σy, Σx2 and Σxy (Σ means "sum up")

Step 3:	Calculate Slope m:

m = (N(Σxy) − (Σx) (Σy))/(N(Σx^2) − (Σx)^2)

(N is the number of points.)

Step 4:	Calculate Intercept a:

a = (Σy) - b(Σx)

Step 5: Assemble the equation of a line

y = mx + a

Source: https://www.mathsisfun.com/data/least-squares-regression.html
#Moving Average (MA)
Source: https://en.wikipedia.org/wiki/Moving-average_model
#Least Squares Regression vs Autoregression (AR)
What is the difference between Least Squares Regression and Autoregression (AR)?

Source: https://machinelearningmastery.com/autoregression-models-time-series-forecasting-python/ 


In [0]:
!pip install --upgrade jhtalib
apikey = 'Apikey 9fa3addd7c29c1da3a51de2d10628c4d6c3747e2577c79ea047a85068708d66f'


Requirement already up-to-date: jhtalib in /usr/local/lib/python3.6/dist-packages (20190319.0)


In [0]:

from pandas import Series
from pandas import DataFrame
from pandas import concat
import pandas as pd
from matplotlib import pyplot
#series = roc_list #Series.from_csv('EHV_HP.csv', header=0)
d = {'l': roc_list[-20:]}
#values = DataFrame(data=d)
df = pd.DataFrame(data=d)

#values = Series(roc_list)
dataframe = concat([df.shift(1), df], axis=1)
#dataframe.columns = ['t-1', 't+1']
#result = dataframe.corr()
#print(result)
from pandas import Series
from matplotlib import pyplot
from statsmodels.graphics.tsaplots import plot_acf
#series = Series.from_csv('daily-minimum-temperatures.csv', header=0)
plot_acf(df, lags=11)
#autocorrelation_plot(values)
pyplot.show()

Interpretation: With this data we could estimate a model with AR(4)
PAC(4)

Linear Regression 2 different calculations (LSR and SLR) with 2 different outcomes. And calculation of AR. Can you check the AR output?



---



In [0]:
import requests
from datetime import datetime as dt
from pprint import pprint as pp
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import jhtalib as jhta

import numpy


def autocorr1(x,lags):
    '''numpy.corrcoef, partial'''

    corr=[1. if l==0 else numpy.corrcoef(x[l:],x[:-l])[0][1] for l in lags]
    return numpy.array(corr)

def autocorr2(x,lags):
    '''manualy compute, non partial'''

    mean=numpy.mean(x)
    var=numpy.var(x)
    xp=x-mean
    corr=[1. if l==0 else numpy.sum(xp[l:]*xp[:-l])/len(x)/var for l in lags]

    return numpy.array(corr)

def autocorr3(x,lags):
    '''fft, pad 0s, non partial'''

    n=len(x)
    # pad 0s to 2n-1
    ext_size=2*n-1
    # nearest power of 2
    fsize=2**numpy.ceil(numpy.log2(ext_size)).astype('int')

    xp=x-numpy.mean(x)
    var=numpy.var(x)

    # do fft and ifft
    cf=numpy.fft.fft(xp,fsize)
    sf=cf.conjugate()*cf
    corr=numpy.fft.ifft(sf).real
    corr=corr/var/n

    return corr[:len(lags)]

def autocorr4(x,lags):
    '''fft, don't pad 0s, non partial'''
    mean=x.mean()
    var=numpy.var(x)
    xp=x-mean

    cf=numpy.fft.fft(xp)
    sf=cf.conjugate()*cf
    corr=numpy.fft.ifft(sf).real/var/len(x)

    return corr[:len(lags)]

def autocorr5(x,lags):
    '''numpy.correlate, non partial'''
    mean=x.mean()
    var=numpy.var(x)
    xp=x-mean
    corr=numpy.correlate(xp,xp,'full')[len(x)-1:]/var/len(x)

    return corr[:len(lags)]


def testcor():

    y=[0, 2, 1, 3, 2, 4, 3, 5, 4, 6]
    
    y=numpy.array(y).astype('float')

    lags=range(2)
    fig,ax=plt.subplots()

    for funcii, labelii in zip([autocorr1, autocorr2, autocorr3, autocorr4,
        autocorr5], ['np.corrcoef, partial', 'manual, non-partial',
            'fft, pad 0s, non-partial', 'fft, no padding, non-partial',
            'np.correlate, non-partial']):

        cii=funcii(y,lags)
        print(labelii)
        print(cii)
        ax.plot(lags,cii,label=labelii)

    ax.set_xlabel('lag')
    ax.set_ylabel('correlation coefficient')
    ax.legend()
    plt.show()

testcor()

def ACF():
  """
  Auto Correlation Function
  """

def PACF():
  """
  Partial Auto Correlation Function
  """


y=[0, 2, 1, 3, 2, 4, 3, 5, 4, 6]
y0=y[2:10]
y1=y[1:9]
y2=y[0:8]
    
cory0y1=jhta.COR(y0,y1)
pp ('cory0y1')
pp(cory0y1)
cory1y2=jhta.COR(y1,y2)
pp ('cory1y2')
pp(cory1y2)
cory2y0=jhta.COR(y2,y0)
pp ('cory2y0')
pp(cory2y0)




#Smoothing
Finanicial hacker

Source: https://www.financial-hacker.com/trend-delusion-or-reality/
We can use the result of the pacf (the amount of partial correlation function lags with predictive value) to input as smoothing period or the trend

#Relative Volatility Index (RVI) (see below: rvi original)

#Relative Volatility Index Smoothed (RVIS)
Because most indicators measure price change, Dorsey developed the RVI as a confirming indicator that measures the direction of volatility. It is almost identical to the Relative Strength Index (RSI) but uses the standard deviation of high and low prices.

How To Calculate the Relative Volatility Index Smoothed (RVIS) using EMA
RVI = 100 * U / (U + D)
Where:
U = EMA(N*2)-1,N of USD
D = EMA(N*2)-1,N of DSD
USD = If close > close(1) then SD,S else 0
DSD = If close < close(1) then SD,S else 0
S = User selected period for the Standard Deviation of the close (Dorsey suggested 10).
N = User selected smoothing period (Dorsey suggested 14)
(Instead of using Wilder’s Smoothing **we use an EMA** with a period of (N*2)-1 which produces the same result but is faster to calculate.)

Source: http://etfhq.com/blog/2011/02/16/relative-volatility-index/

The Inertia indicator is plotted on a scale of 0 to 100 and its middle level of 50 acts as a threshold separating uptrends from downtrends.

Source: http://www.binarytribune.com/forex-trading-indicators/inertia-indicator/

#Inertia
Donald Dorsey (Stocks & Commodities , V.13:9 (September, 1995): "Refining the Relative Volatility Index")
//@version=3
// Copyright (c) 2019-present, Alex Orekhov (everget)
// Dorsey Inertia script may be freely distributed under the MIT license.
study("Dorsey Inertia", shorttitle="Inertia")

stdevLength = input(title="Standard Deviation Length", type=integer, defval=21)
rviSmoothLength = input(title="RVI Smoothing Length", type=integer, defval=14)
smoothLength = input(title="Inertia Smoothing Length", type=integer, defval=14)

#RVI original
// Relative Volatility Index (1993)
rviOriginal(src, stdevLength, smoothLength) =>
	stdev = stdev(src, stdevLength)
	upSum = ema(change(src) >= 0 ? stdev : 0, smoothLength)
	downSum = ema(change(src) >= 0 ? 0 : stdev, smoothLength)
	100 * upSum / (upSum + downSum)

rvi = avg(rviOriginal(high, stdevLength, rviSmoothLength), rviOriginal(low, stdevLength, rviSmoothLength))
inertia = linreg(rvi, smoothLength, 0)

inertiaColor = inertia > 50 ? #0ebb23 : red

plot(inertia, title="Inertia", linewidth=2, color=inertiaColor, transp=0)
hline(50, title="Middle Level", linestyle=dotted)

Source: https://www.tradingview.com/script/bzxmXFGd-Dorsey-Inertia/

////////////////////////////////////////////////////////////
//  Copyright by HPotter v1.0 23/05/2017
// The inertia indicator measures the market, stock or currency pair momentum and 
// trend by measuring the security smoothed RVI (Relative Volatility Index). 
// The RVI is a technical indicator that estimates the general direction of the 
// volatility of an asset.
// The inertia indicator returns a value that is comprised between 0 and 100. 
// Positive inertia occurs when the indicator value is higher than 50. As long as 
// the inertia value is above 50, the long-term trend of the security is up. The inertia 
// is negative when its value is lower than 50, in this case the long-term trend is 
// down and should stay down if the inertia stays below 50.
////////////////////////////////////////////////////////////
study(title="Inertia Indicator", shorttitle="Inertia")
Period = input(10, minval=1)
Smooth = input(14, minval=1)
hline(50, color=green, linestyle=line)
xPrice = close
StdDev = stdev(xPrice, Period)
d = iff(close > close[1], 0, StdDev)
u = iff(close > close[1], StdDev, 0)
nU = (13 * nz(nU[1],0) + u) / 14
nD = (13 * nz(nD[1],0) + d) / 14
nRVI = 100 * nU / (nU + nD)
nRes = ema(nRVI, Smooth)
plot(nRes, color=red, title="Inertia")

Source: https://www.tradingview.com/script/JKJGpmuX-Inertia-Indicator/

#RVI and MACD
https://www.researchgate.net/profile/Dejan_Eric/publication/228820374_Application_of_MACD_and_RVI_indicators_as_functions_of_investment_strategy_optimization_on_the_financial_market/links/546d40b40cf2193b94c58100/Application-of-MACD-and-RVI-indicators-as-functions-of-investment-strategy-optimization-on-the-financial-market.pdf?origin=publication_detail

In [0]:
import requests
from datetime import datetime as dt
from pprint import pprint as pp
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import jhtalib as jhta


def AR(df, n, price='Close'):

  test1=jhta.COR([1, 2, 3, 4, 5, 6, 7, 8, 9],[0, 1, 2, 3, 4, 5, 6, 7, 8])
  pp(test1)
  end = len(df[price]) - n
  x_list = list(range(len(df[price][0:end])))
  y_list = df[price][0:end]
  y1_list = df[price][1:end+1]
  y2_list = df[price][2:end+2]
  b1 = jhta.COV(x_list, y_list) / jhta.VARIANCE({'x': x_list}, len(x_list), 'x')[-1]
  b0 = jhta.MEAN({'y': y_list}, len(y_list), 'y')[-1] - b1 * jhta.MEAN({'y': x_list}, len(x_list), 'x')[-1]
  ar_dict = {'train': [], 'test': []}
  print (jhta.COV(x_list, y_list))
  print (jhta.VARIANCE({'x': x_list}, len(x_list), 'x')[-1])
  print (jhta.MEAN({'x': x_list}, len(x_list), 'x')[-1])
  print (jhta.MEAN({'y': y_list}, len(y_list), 'y')[-1])
  print (b1)
  print (b0)
  i = 0
  while i < len(df[price]):
    y = b0 + b1 * i # Dit is OLS
    if i < len(df[price]) - n:
      ar_dict['train'].append(y)
      ar_dict['test'].append(float('NaN'))
    else:
      ar_dict['train'].append(float('NaN'))
      ar_dict['test'].append(y)
    i += 1
  return ar_dict

def INERTIA(df):
  """
  Inertia

  source: https://www.fmlabs.com/reference/Inertia.htm
  """
  
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] #tijdschaal
y = [0, 2, 1, 3, 2, 4, 3, 5, 4, 6] #input 'test' serie

cor = jhta.COR(x, y)
print (cor)
pcor = jhta.PCOR(x, y)
print (pcor)

predictions_int = 0

lsr_list = jhta.LSR({'y': y}, 'y', predictions_int)
slr_list = jhta.SLR({'y': y}, 'y', predictions_int)

n = 2

#ar_dict = AR({'y': y}, n, 'y')

y.extend([float('NaN')] * predictions_int)
x = list(range(len(y)))

print (lsr_list)
print (slr_list)
#print (ar_dict['train'])
#print (ar_dict['test'])

plt.figure(1, figsize=(30, 20))

plt.subplot(311)
plt.title('Time / Price / Ratio')
plt.xlabel('Time')
plt.ylabel('Price')
plt.grid(True)
plt.plot(x, y, color='blue')
plt.plot(x, lsr_list, color='red')
plt.legend(['Y', 'Least Squares Regression'], loc='upper left')
#plt.yscale('log')

plt.subplot(312)
plt.title('Time / Price / Ratio')
plt.xlabel('Time')
plt.ylabel('Price')
plt.grid(True)
plt.plot(x, y, color='blue')
plt.plot(x, slr_list, color='red')
plt.legend(['Y', 'Simple Linear Regression'], loc='upper left')
#plt.yscale('log')

plt.subplot(313)
plt.title('Time / Price / Ratio')
plt.xlabel('Time')
plt.ylabel('Price')
plt.grid(True)
plt.plot(x, y, color='blue')
#plt.plot(x, ar_dict['train'], color='red')
#plt.plot(x, ar_dict['test'], color='orange')
plt.legend(['Y', 'AutoRegression'], loc='upper left')
#plt.yscale('log')

plt.show()





#Candlestick
source: http://www.candlestickforum.com/PPF/Parameters/16_263_/candlestick.asp
source: https://www.tradingview.com/script/vcsWo8mh-Candlestick-Patterns-Identified-updated-3-11-15/
source: https://www.tradeciety.com/understand-candlesticks-patterns/
source: https://www.mql5.com/en/articles/5576
source: https://www.mql5.com/en/articles/101
source: https://www.ta-lib.org/hdr_dw.html for converting pattern recognition from C to Python3




In [0]:
!pip install --upgrade jhtalib
apikey = 'Apikey 9fa3addd7c29c1da3a51de2d10628c4d6c3747e2577c79ea047a85068708d66f'


In [0]:
# http://www.candlestickforum.com/PPF/Parameters/16_263_/candlestick.asp
# https://www.tradingview.com/script/vcsWo8mh-Candlestick-Patterns-Identified-updated-3-11-15/
# https://www.tradeciety.com/understand-candlesticks-patterns/
# https://www.mql5.com/en/articles/5576
# https://www.mql5.com/en/articles/101
# https://www.ta-lib.org/hdr_dw.html for converting pattern recognition from C to Python3




import requests
from datetime import datetime as dt
from pprint import pprint as pp
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import jhtalib as jhta


url = 'https://min-api.cryptocompare.com/data/histoday?fsym=ETH&tsym=BTC&allData=true'
headers = {'authorization': apikey}
r = requests.get(url, headers=headers).json()
#print (len(r['Data']))
#pp (r)

df = {'datetime': [], 'Open': [], 'High': [], 'Low': [], 'Close': [], 'Volume': []}
i = 0
while i < len(r['Data']):
#  df['datetime'].append(i)
#  df['datetime'].append(r['Data'][i]['time'])
  df['datetime'].append(dt.fromtimestamp(r['Data'][i]['time']))
  df['Open'].append(float(r['Data'][i]['open']))
  df['High'].append(float(r['Data'][i]['high']))
  df['Low'].append(float(r['Data'][i]['low']))
  df['Close'].append(float(r['Data'][i]['close']))
  df['Volume'].append(int(r['Data'][i]['volumefrom']))
  i += 1

jhta.DF2CSV(df, 'ETHBTC.csv')
#jhta.INFO(df)


def DOJI(df):
  """
  Doji
  """
  cdl_list = []
  i = 0
  while i < len(df['Close']):
    o = df['Open'][i]
    h = df['High'][i]
    l = df['Low'][i]
    c = df['Close'][i]

    cdl = 0
    
#    if o == c:
    if abs(o - c) <= (h - l) * .1:
      cdl = 1

    cdl_list.append(cdl)
    i += 1
  return cdl_list

def ENGULFING(df):
  """
  Engulfing
  """
  cdl_list = []
  i = 0
  while i < len(df['Close']):
    o = df['Open'][i]
    h = df['High'][i]
    l = df['Low'][i]
    c = df['Close'][i]
    o1 = df['Open'][i - 1]
    h1 = df['High'][i - 1]
    l1 = df['Low'][i - 1]
    c1 = df['Close'][i - 1]
    
    cdl = 0
    
    # bullish:
    if (o1 > c1) and (c > o) and (c >= o1) and (c1 >= o) and ((c - o) > (o1 - c1)):
      cdl = 1
    
    # bearish:
    if (c1 > o1) and (o > c) and (o >= c1) and (o1 >= c) and ((o - c) > (c1 - o1)):
      cdl = -1
    
    cdl_list.append(cdl)
    i += 1
  return cdl_list

def HAMMER(df):
  """
  Hammer
  """
  cdl_list = []
  i = 0
  while i < len(df['Close']):
    o = df['Open'][i]
    h = df['High'][i]
    l = df['Low'][i]
    c = df['Close'][i]
    
    cdl = 0
    
    # hammer:
    if (h - l) > (3 * (o - c)) and ((c - l) / (.001 + h - l) > .6) and ((o - l) / (.001 + h - l) > .6):
      cdl = 1
    
    # inverted hammer:
    if (h - l) > (3 * (o - c)) and ((h - c) / (.001 + h - l) > .6) and ((h - o) / (.001 + h - l) > .6):
      cdl = 1
    
    cdl_list.append(cdl)
    i += 1
  return cdl_list

  
#pp (df['Close'])
#pp (jhta.RVI(df, 10))
#pp (jhta.RMI(df, 10))
#pp (DOJI(df))
pp (ENGULFING(df))
#pp (HAMMER(df))




  
#Augmented Dickey Fuller stationair test
  from statsmodels.tsa.stattools import adfuller
  def test_stationarity(timeseries):

#Determing rolling statistics
      rolmean = pd.rolling_mean(timeseries, window=12)
      rolstd = pd.rolling_std(timeseries, window=12)

      #Plot rolling statistics:
      orig = plt.plot(timeseries, color='blue',label='Original')
      mean = plt.plot(rolmean, color='red', label='Rolling Mean')
      std = plt.plot(rolstd, color='black', label = 'Rolling Std')
      plt.legend(loc='best')
      plt.title('Rolling Mean & Standard Deviation')
      plt.show(block=False)

      #Perform Dickey-Fuller test:
      print 'Results of Dickey-Fuller Test:'
      dftest = adfuller(timeseries, autolag='AIC')
      dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
      for key,value in dftest[4].items():
          dfoutput['Critical Value (%s)'%key] = value
      print dfoutput
  
# Autocorrelation

  def autocorrelation (x) :
  
    """
    Compute the autocorrelation of the signal, based on the properties of the
    power spectral density of the signal.
    """
    xp = x-np.mean(x)
    f = np.fft.fft(xp)
    p = np.array([np.real(v)**2+np.imag(v)**2 for v in f])
    pi = np.fft.ifft(p)
    return np.real(pi)[:x.size/2]/np.sum(xp**2)
    
Source: https://stackoverflow.com/questions/643699/how-can-i-use-numpy-correlate-to-do-autocorrelation

